In [1]:

import pandas as pd
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)

import altair as alt


import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append("..")
from network_analysis.load_datasets import get_updated_shxco_data



In [2]:
members_df, books_df, borrow_events, events_df = get_updated_shxco_data(
    get_subscription=False)


In [3]:
ex_books_df = books_df[books_df.exceptional_types.isna() == False].assign(
    exceptional_types=books_df['exceptional_types'].str.split(',')).explode('exceptional_types')
grouped_books = ex_books_df.groupby(
    'exceptional_types').size().reset_index(name='counts')
grouped_books['type'] = 'books'

ex_members_df = members_df[members_df.exceptional_types.isna() == False].assign(exceptional_types=members_df['exceptional_types'].str.split(
    ',')).explode('exceptional_types')
grouped_members = ex_members_df.groupby(
    'exceptional_types').size().reset_index(name='counts')
grouped_members['type'] = 'members'

ex_events_df = events_df[events_df.exceptional_types.isna() == False].assign(exceptional_types=events_df['exceptional_types'].str.split(
    ',')).explode('exceptional_types')
grouped_events = ex_events_df.groupby(
    'exceptional_types').size().reset_index(name='counts')
grouped_events['type'] = 'events'


In [4]:
grouped_df = pd.concat([grouped_books, grouped_members, grouped_events])
grouped_df.loc[grouped_df['exceptional_types'] ==
               'sunday_shopers', 'exceptional_types'] = 'Sunday Shoppers'
grouped_df.loc[grouped_df['exceptional_types'] ==
               'longterm_borrows', 'exceptional_types'] = 'Longterm Borrows'
grouped_df.loc[grouped_df['exceptional_types'] == 'overborrows',
               'exceptional_types'] = 'Borrows Beyond Subscription'

grouped_df.loc[grouped_df['exceptional_types'] ==
               'post1942_events', 'exceptional_types'] = 'Borrows After 1942'

grouped_df.loc[grouped_df['exceptional_types'] ==
               'unknown_borrows', 'exceptional_types'] = 'Borrow Status Unknown'

grouped_df.loc[grouped_df['exceptional_types'] ==
               'missing_events', 'exceptional_types'] = 'Borrow Status Missing'


In [5]:

chart = alt.Chart(grouped_df).mark_bar().encode(
    y=alt.Y('type:O', sort=alt.EncodingSortField(
        field='counts', op='sum', order='descending'), title=''),
    x=alt.X('counts:Q', title=''),
    color=alt.Color('type:N', legend=alt.Legend(title='Library Dataset'), sort=alt.EncodingSortField(field='counts', op='sum', order='descending')),
).properties(
    width=100,
    height=100,
).facet(
    facet=alt.Facet('exceptional_types:O', title=None),
    columns=3,
    title='Rate of Exceptional Behavior in the Library (Scale Independent)'
).resolve_scale(x='independent')
chart
# chart.save('./visualizations/test.png')


alt.FacetChart(...)

In [6]:
len(events_df[events_df.exceptional_types.isna() == False]), len(
    events_df), len(events_df[events_df.exceptional_types.isna() == True])


(6927, 34357, 27430)